In [6]:
import os
import random
import sys
import time
from urllib.parse import urlparse
from selenium.common.exceptions import ElementNotInteractableException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd

In [7]:
# Open Webdriver
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome('chromedriver.exe', options=options)
driver.set_window_size(800,800)
# Getting login details
file = open('config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]
globalPauseTime = 3

# LinkedIn login page
driver.get('https://www.linkedin.com/uas/login')

# Insert Username
elementID = driver.find_element_by_id('username')
elementID.send_keys(username)

# Insert Password
elementID = driver.find_element_by_id('password')
elementID.send_keys(password)
time.sleep(globalPauseTime)

# Submit
elementID.submit()


In [18]:
# Reading excel file with LinkedIn Accounts
dataLocation = "accounts.xlsx"
df = pd.read_excel(dataLocation)
# Check number of rows
rows = len(df)

In [19]:
# Looking for each account
for row in range(rows):
    studentAccount = df.loc[row][0]
    searchURL = studentAccount
    driver.get(searchURL)
     
    for i in range(1):
        wait= WebDriverWait(driver, 5)
        time.sleep(7)
        connection = driver.find_element_by_class_name('dist-value').get_attribute("innerHTML").strip()        
        if(connection == '1st'):
            name = driver.find_element_by_class_name('text-heading-xlarge').get_attribute("innerHTML")
            job_title = driver.find_element_by_class_name('text-body-medium break-words'.replace(' ', '.')).get_attribute("innerHTML").strip()
            location = driver.find_element_by_class_name('text-body-small inline t-black--light break-words'.replace(' ', '.')).get_attribute("innerHTML").strip()
            time.sleep(4)            
            contactDetails = driver.find_element_by_link_text('Contact info')
            contactDetails.click()
            time.sleep(5)
            try:
                email_section =  driver.find_element(By.XPATH, "//section[@class='pv-contact-info__contact-type ci-email']")
                email = email_section.find_element_by_class_name('pv-contact-info__contact-link link-without-visited-state t-14'.replace(' ', '.')).get_attribute("innerHTML").strip()
            except:
                email = 'no email'
            try:
                company_link = driver.find_element(By.XPATH, "//*[@data-field='experience_company_logo']").get_attribute("href")
                company_name = driver.find_element(By.XPATH, "//a[@data-field='experience_company_logo']//span[@class='t-14 t-normal']//span[1]").text.strip()
                company_name2 = driver.find_element_by_xpath("//*/ul/li/div/div[2]/div[1]/a/div/span/span[1]").text.strip()
                time.sleep(3)
                no_company_link = 'https://www.linkedin.com/search/results/all/'
                if(company_link != None and no_company_link in company_link):
                    print('no company details')
                    employee_number ='No company details'
                else:
                    driver.get(company_link)
                    time.sleep(4)
                    people = driver.find_element_by_link_text("People")
                    people.click()
                    time.sleep(5)
                    employee_number = driver.find_element_by_xpath("//main[@id='main']/div[@class='org-grid__content-height-enforcer']/div/div[@class='artdeco-card pb2']/div[@class='display-flex full-width justify-space-between align-items-center pt5 ph5']/span[@class='t-20 t-black t-bold']").text
                    
            except:
                company_link = 'No company Link'
                company_name = 'No company Name'
                company_name2 = 'No company Name'
                

            df.loc[row, 'jobtitle'] = job_title
            df.loc[row, 'location'] = location
            df.loc[row, 'companylink'] = company_link
            df.loc[row, 'companyname'] = company_name
            df.loc[row, 'companyname2'] = company_name2
            df.loc[row, 'employeeNumber'] = employee_number
            df.loc[row, 'email'] = email
            df.to_excel(dataLocation, index=False)
            print(company_link, " ", email, " ", company_name, " ", company_name2)
            print(employee_number)
            print(name, connection, job_title, location)
        else:
            name = driver.find_element_by_class_name('text-heading-xlarge').get_attribute("innerHTML")
            job_title = driver.find_element_by_class_name('text-body-medium break-words'.replace(' ', '.')).get_attribute("innerHTML").strip()
            location = driver.find_element_by_class_name('text-body-small inline t-black--light break-words'.replace(' ', '.')).get_attribute("innerHTML").strip()
            time.sleep(4)
            try:
                company_link = driver.find_element(By.XPATH, "//*[@data-field='experience_company_logo']").get_attribute("href")
                company_name = driver.find_element(By.XPATH, "//a[@data-field='experience_company_logo']//span[@class='t-14 t-normal']//span[1]").text.strip()
                company_name2 = driver.find_element_by_xpath("//*/ul/li/div/div[2]/div[1]/a/div/span/span[1]").text.strip()
                no_company_link = "https://www.linkedin.com/search/results/all/"
                if(company_link != None and no_company_link in company_link):
                    print('no company details')
                    employee_number ='No company details'
                else:
                    driver.get(company_link)
                    time.sleep(4)
                    people = driver.find_element_by_link_text("People")
                    people.click()
                    time.sleep(5)
                    employee_number = driver.find_element_by_xpath("//main[@id='main']/div[@class='org-grid__content-height-enforcer']/div/div[@class='artdeco-card pb2']/div[@class='display-flex full-width justify-space-between align-items-center pt5 ph5']/span[@class='t-20 t-black t-bold']").text            
            except:
                company_link = 'No company Link'
                company_name = 'No company Name'
                company_name2 = 'No company Name'
                employee_number = 'no employee data'            
            #company_name = company.find_element(By.XPATH, "//span[2][@aria-hidden='true']").get_attribute("innerHTML")
            df.loc[row, 'jobtitle'] = job_title
            df.loc[row, 'location'] = location
            df.loc[row, 'companylink'] = company_link
            df.loc[row, 'companyname'] = company_name
            df.loc[row, 'companyname2'] = company_name2
            df.loc[row, 'employeeNumber'] = employee_number

            df.to_excel(dataLocation, index=False)
            print(company_link, " ", company_name, " ", company_name2)
            print(employee_number)
            print(name, connection, job_title, location)

no company details
https://www.linkedin.com/search/results/all/?keywords=Care+World   Care World   Care Manager
No company details
Susan Jordan 3rd Care Manager at Care World Ireland
https://www.linkedin.com/company/300034/   Career Choices Series & My10yearPlan.com by Academic Innovations · Full-time   Salesforce Administrator
33 employees
Mike Quill 2nd Salesforce Administrator | 6 x Salesforce Certified | CTO of AppSelekt | Multimedia Developer and Designer San Diego County, California, United States
https://www.linkedin.com/company/1105836/   Carton House, A Fairmont Managed Hotel   Director of Operations - Golf
143 employees
Jason Totos 3rd Operations Director: Promoting growth through flexible strategy and cooperative systems. Ireland
https://www.linkedin.com/company/137277/   3 yrs 10 mos   CarTrawler
435 employees
Andrew O'Farrell 3rd Partner Relationship Manager at CarTrawler County Dublin, Ireland
https://www.linkedin.com/company/137277/   CarTrawler   1 mutual group
435 empl

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//section[@class='pv-contact-info__contact-type ci-email']"}
  (Session info: chrome=96.0.4664.93)


In [56]:
#FUNCTIONS

def scroll_window():
    height = driver.execute_script("return document.documentElement.scrollHeight")
    scrollPauseTime = 3
    driver.execute_script("window.scrollTo(0, " + str(height/2) + ");")
    time.sleep(scrollPauseTime-1)
    driver.execute_script("window.scrollTo(0, " + str(height) + ");")
    time.sleep(scrollPauseTime+1)
    driver.execute_script("window.scrollTo(0, 0); ")        

def get_student_details():
    name = driver.find_element_by_class_name('text-heading-xlarge').get_attribute("innerHTML")
    job_title = driver.find_element_by_class_name('text-body-medium break-words'.replace(' ', '.')).get_attribute("innerHTML").strip()
    location = driver.find_element_by_class_name('text-body-small inline t-black--light break-words'.replace(' ', '.')).get_attribute("innerHTML").strip()
    time.sleep(4)
    return name, job_title, location

def get_company_details():
    try:
        company_link = driver.find_element(By.XPATH, "//*[@data-field='experience_company_logo']").get_attribute("href")
        company_name = driver.find_element(By.XPATH, "//a[@data-field='experience_company_logo']//span[@class='t-14 t-normal']//span[1]").text.strip()
        company_name2 = driver.find_element_by_xpath("//*/ul/li/div/div[2]/div[1]/a/div/span/span[1]").text.strip()
        time.sleep(3)
        no_company_link = 'https://www.linkedin.com/search/results/all/'
        if(company_link != None and no_company_link in company_link):
            print('no company details')
        else:
            driver.get(company_link)
            time.sleep(4)
            people = driver.find_element_by_link_text("People")
            people.click()
            time.sleep(5)
            employee_number = driver.find_element_by_xpath("//main[@id='main']/div[@class='org-grid__content-height-enforcer']/div/div[@class='artdeco-card pb2']/div[@class='display-flex full-width justify-space-between align-items-center pt5 ph5']/span[@class='t-20 t-black t-bold']").text            
 
    except:
        company_link = 'No company Link'
        company_name = 'No company Name'
        company_name2 = 'No company Name'
        employee_number ='No company details'
    return company_link, company_name, company_name2, employee_number

def append_details_to_excell():
    df.loc[row, 'jobtitle'] = job_title
    df.loc[row, 'location'] = location
    df.loc[row, 'companylink'] = company_link
    df.loc[row, 'companyname'] = company_name
    df.loc[row, 'companyname2'] = company_name2
   
    

In [ ]:
company_name = driver.find_element(By.XPATH, "//a[@data-field='experience_company_logo']//span[@class='t-14 t-normal']").text.strip()
print(company_name)
experience = driver.find_elements_by_xpath("//h2[@class='pvs-header__title text-heading-large']/span[2]")

for ex in experience:
    print(ex.text)
    if(ex.text == 'Experience'):
        print('it has experience')
    else:
        print('no experience')

In [7]:
company_link = driver.find_element(By.XPATH, "//*[@data-field='experience_company_logo']").get_attribute("href")
print(company_link)
driver.get(company_link)

https://www.linkedin.com/company/3334817/


In [20]:

people = driver.find_element_by_link_text("People")
people.click()
employee_number = driver.find_element_by_xpath("//main[@id='main']/div[@class='org-grid__content-height-enforcer']/div/div[@class='artdeco-card pb2']/div[@class='display-flex full-width justify-space-between align-items-center pt5 ph5']/span[@class='t-20 t-black t-bold']").text
print(employee_number)
experience = driver.find_elements_by_xpath("//h2[@class='pvs-header__title text-heading-large']/span[2]")

11 employees
